In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
import seaborn as sns
from gensim.utils import simple_preprocess
import numpy as np
import gensim.parsing.preprocessing as pp
from gensim.models import Word2Vec, KeyedVectors
import gensim

# Постановка задачи и загрузка данных

In [2]:
!gdown 1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi

Downloading...
From (original): https://drive.google.com/uc?id=1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi
From (redirected): https://drive.google.com/uc?id=1onuImgIOYhiqwnBuFsDnvepD2R5Gkebi&confirm=t&uuid=8f3d8787-64b7-41fc-990a-33f974676013
To: /content/df_up_courses_202308.csv
100% 1.88G/1.88G [00:23<00:00, 79.4MB/s]


Из информации о дисциплине у нас есть следующая:
1. course_id - номер дисциплины (может повторяться)
2. course_title - название дисциплины
3. description - краткое описание дисциплины
4. section - разделы дисциплины
5. topic - темы, входящие в раздел дисциплины
6. qualification - уровень образования (бакалавриат, магистратура, специалитет)
7. unit_id - айди факульета
8. up_id - айди учебного плана (образовательной программы)
9. up_title - название учебного плана (образовательной программы)
10. start_year - год набора
11. module_name - название модуля, в который входит дисциплина

В рамках практики по обработке текста поработаем с реальными данными, содержательно близкими к данным с первой практики, - из Конструктора ОП. В этот раз поработаем с описанием дисциплин с помощью языковых моделей.


Задача на практику - предобработать датасет:
1. получить текстовое описание каждой дисциплины датасета
2. выполнить предобработку текстовых данных
3. построить для дисциплины вектор на основе word2vec
4. потестировать текстовые представления

Задач не много, но почти все затратные по времени.

## 3.1 Предобработка датасета

Преобразуйте датасет так, чтобы одна строка описывала одну дисциплину в учебном плане. Отдельным столбцом должен храниться весь текст, который описывает дисциплину: description, section, topic. Без дубликатов содержания разделов и тем внутри дисциплины.

In [3]:
df = pd.read_csv("/content/df_up_courses_202308.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df

,course_id,course_title,description,section,topic,qualification,unit_id,up_id,up_title,start_year,module_name
0,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,6905,Программирование и интернет-технологии,2018,"Специализация ""Информационные системы и технол..."
1,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,6938,Программирование и интернет-технологии,2019,"Специализация ""Информационные системы и технол..."
2,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,"Понятия ИС, КИС, АСУ, АСУП. ИС как инструмент ...",bachelor,14.0,7014,Программирование и интернет-технологии,2020,"Специализация ""Разработка информационных систе..."
3,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,Различные методики управления организацией,bachelor,14.0,6905,Программирование и интернет-технологии,2018,"Специализация ""Информационные системы и технол..."
4,2811,Архитектура информационных систем,Целью освоения дисциплины «Архитектура информа...,Функциональная и информационная архитектуры,Различные методики управления организацией,bachelor,14.0,6938,Программирование и интернет-технологии,2019,"Специализация ""Информационные системы и технол..."
...,...,...,...,...,...,...,...,...,...,...,...
1039704,23816,Методы и технологии генеративного и композитно...,Глубинные генеративные модели широко использую...,Генеративные несостязательные модели искусстве...,Вариационные автокодировщики.,master,49.0,7177,Глубокое обучение и генеративный искусственный...,2022,Фронтирные технологии ИИ
1039705,23816,Методы и технологии генеративного и композитно...,Глубинные генеративные модели широко использую...,Генеративные несостязательные модели искусстве...,"Модели, основанные на энергии как альтернатива...",master,49.0,7177,Глубокое обучение и генеративный искусственный...,2022,Фронтирные технологии ИИ
1039706,23816,Методы и технологии генеративного и композитно...,Глубинные генеративные модели широко использую...,Генеративные несостязательные модели искусстве...,Нормализующие потоки и диффузионные модели как...,master,49.0,7177,Глубокое обучение и генеративный искусственный...,2022,Фронтирные технологии ИИ
1039707,25014,Современные проблемы машинного обучения,В этом курсе мы сначала рассмотрим некоторые х...,Условия проведения семинара,Условия проведения семинара,master,49.0,7177,Глубокое обучение и генеративный искусственный...,2022,Обязательные дисциплины


In [4]:
df.loc[:, 'description'] = df['description'].astype(str)
df.loc[:, 'section'] = df['section'].astype(str)
df.loc[:, 'topic'] = df['topic'].astype(str)

grouped = df.groupby(['course_id', 'course_title', 'qualification', 'unit_id', 'up_id', 'up_title', 'start_year', 'module_name'])
df_text = grouped.agg({
    'description': lambda x: ' '.join(pd.unique(x)),
    'section': lambda x: ' '.join(pd.unique(x)),
    'topic': lambda x: ' '.join(pd.unique(x))
}).reset_index()
df_text['text'] = df_text['description'] + ' ' + df_text['section'] + ' ' + df_text['topic']
df_text.drop(columns=['section', 'topic', 'description'], inplace=True)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df_text.to_csv("/content/drive/MyDrive/df/df_prepared.csv", encoding='utf-8', index=False)

In [ ]:
df_prepared = pd.read_csv("/content/drive/MyDrive/df/df_prepared.csv", encoding='utf-8')
df_prepared.head()

Преобразования выполняются не слишком быстро, рекомендую сохраниться.

## 3.2 Предобработка описания дисциплины

Пора поработать со столбцом с текстовым содержанием дисциплины. Минимально понадобятся следующие шаги предобработки:
1. Токенизация
2. Удаление пунктуации
3. Удаление лишних пробелов внутри текста
4. Лемматизация

Чтобы не слишком углубляться в предобработку, рекомендую воспользоваться реализованными в gensim методами. Примеры можно найти в ноутбуке по ссылке: https://colab.research.google.com/drive/1Lxe0_dPxF5_nNC1hnHTRc01rnMmUVCXa?usp=sharing

In [ ]:
def preprocess_text(text):
    filters = [
        pp.strip_tags,
        pp.strip_numeric,
        pp.remove_stopwords,
        pp.strip_multiple_whitespaces,
        pp.strip_punctuation,
        pp.stem_text
    ]
    preprocessed_text = pp.preprocess_string(text, filters)
    return ' '.join(preprocessed_text)
df_prepared['preprocessed'] = df_prepared['text'].apply(lambda x: preprocess_text(x))

In [ ]:
df_prepared.to_csv('/content/drive/MyDrive/df/df_preprocessed.csv', encoding='utf-8', index=False)

In [ ]:
df_preprocessed = pd.read_csv("/content/drive/MyDrive/df/df_preprocessed.csv", encoding='utf-8')
df_preprocessed.head()

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,preprocessed
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...


In [ ]:
!pip install pymorphy2
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def lemmatize_text(sentence):
    words = word_tokenize(sentence)
    lemmas = []
    for word in words:
        if len(word) >= 4:
            lemma = str(morph.parse(word)[0].normal_form)
            lemmas.append(lemma)
    return lemmas

df_preprocessed['norm'] = df_preprocessed['preprocessed'].apply(lemmatize_text)

In [ ]:
df_preprocessed.to_csv('/content/drive/MyDrive/df/df_normalized.csv', encoding='utf-8', index=False)

In [17]:
df_normalized = pd.read_csv("/content/drive/MyDrive/df/df_normalized.csv", encoding='utf-8')
df_normalized.head()

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,preprocessed,norm
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '..."
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '..."
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '..."
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '..."
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '..."


Лемматизация выполняется сравнительно долго, поэтому нужно обязательно в этот момент сохранить датасет. Также этот датасет **понадобится для выполненения задания 4**.

## 3.3 word2vec

In [ ]:
df_normalized['norm'] = df_normalized['norm'].head(3000)

In [21]:
import ast
df_normalized['norm'] = df_normalized['norm'].apply(ast.literal_eval)

In [ ]:
# обучим на своих данных модель word2vec
%%time
model = Word2Vec(df_normalized['norm'],                      # данные
                 min_count=1,               # минимальная частота слова в корпусе
                 vector_size=300,           # размер вектора для слова
                 workers=4,                 # количество потоков
                 window=10)                 # окно, в которое попадает слово

In [23]:
model_path = "/content/model" # будет удобно сохранить модель
model.save(model_path)

In [24]:
# так выглядит один вектор
model.wv.get_vector("курс")

array([ 0.08228776,  0.50606996,  0.5300969 ,  0.3403675 ,  0.6098967 ,
        1.8880037 ,  0.64260834, -0.22132897, -0.1321848 ,  0.13279788,
        0.27680784, -0.7629601 , -0.62841684, -0.74505544,  0.33622673,
       -0.17399953, -0.32846594,  0.58758163,  0.22222848,  0.34580606,
        0.39432672, -0.396206  , -0.12736754, -0.39878252, -0.66797763,
        0.27302346,  0.40033913, -1.0741704 ,  0.61080885, -1.4954796 ,
        0.48304123,  0.30848786, -1.7923585 , -0.01394461, -0.26481736,
       -0.7866613 ,  0.75899833,  0.43421966,  0.36357492, -0.52012026,
        0.4960208 , -1.7184049 ,  1.294258  ,  0.72088784, -0.22494592,
        0.5122891 , -0.11068035,  0.17197916,  0.35351133,  0.14328864,
       -0.02012383, -0.7569436 ,  1.2432095 ,  0.29677817, -0.03471873,
        0.7141187 , -0.7665687 , -0.34844497, -0.29169786,  0.094203  ,
       -0.7405061 ,  0.39892858,  0.96603143, -0.30534345, -0.5990335 ,
        1.6671833 ,  2.0098078 , -0.5547569 , -0.5756419 ,  0.12

In [25]:
# слова, максимально близкие к "машинному обучению"
model.wv.most_similar(["обучение", "машинный"])

[('обозначение', 0.7600986957550049),
 ('окончание', 0.7310367226600647),
 ('подкрепление', 0.7130774855613708),
 ('облачный', 0.660179615020752),
 ('нейронный', 0.6411392688751221),
 ('иметь', 0.619721531867981),
 ('быть', 0.6180518865585327),
 ('подход', 0.5750454664230347),
 ('постановка', 0.572999894618988),
 ('туманный', 0.5612046122550964)]

In [27]:
def embed(tokens, w2v_model):
  summary = np.zeros(w2v_model.vector_size)
  count = 0
  for token in tokens:
      # print(token)
      if token in w2v_model.wv.index_to_key:
          count += 1
          summary += w2v_model.wv.get_vector(token)
  return summary / count if count > 0 else summary

df_normalized["embedding"] = df_normalized.norm.apply(lambda x: embed(x, model))

In [29]:
df_normalized.to_csv('/content/df_final_lab3.csv', encoding='utf-8', index=False)

In [30]:
df_final = pd.read_csv('/content/df_final_lab3.csv')
df_final.head()

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,preprocessed,norm,embedding
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 0.07638274 0.15438885 0.2027531 0.150200...
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 0.07638274 0.15438885 0.2027531 0.150200...
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 0.07638274 0.15438885 0.2027531 0.150200...
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 0.07638274 0.15438885 0.2027531 0.150200...
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 0.07638274 0.15438885 0.2027531 0.150200...


Не забываем сохраниться!

## 3.4 Тестирование текстовых представлений

С предобработанными данными попытаемся решить следующие задачи:

1. Найти дисциплины, которые выбиваются из учебного плана. Для этого построим эмбеддинг учебного плана (усредним эмбеддинги всех его дисциплин) и найдем дисциплины, у которых минимальное косинусное сходство с эмбеддингом учебного плана.
2. Оценить, насколько удачно будет генерировать содержание дисциплины LLM. Структура описания дисциплины не важна. Важно содержание. Сравнивать будем эмбеддинги через косинусное сходство.

### 3.4.1 Поиск "лишних" дисциплин

Постройте эмбеддинги для учебных планов по тому же принципу, по которому строили эмбеддинги для дисциплин. Затем проранжируйте все дисциплины в учебном плане по сходству с эмбеддингом плана. Выберите порог для значения сходства, ниже которого дисциплина считается "лишней".


Не нужно оценивать абсолютно все учебные планы из датасета. Можно выбрать около 10 случайных планов или тех, которые вам интересны. И оценить только их.

Обратите внимание, что типичнее для вашей выборки: малое или большое количество "лишних" дисциплин?

Проведите эксперимент: уберите из учебных планов все дисциплины, которые есть в каждом из них. Найдите "лишние" после удаления одинаковых. Оцените результат.

In [32]:
df_final['embedding'] = df_final['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))

In [34]:
grouped = df_final.groupby('up_id').agg(emb_up=('embedding', 'mean')).reset_index()
grouped

,up_id,emb_up
0,6859,"[-0.1489323275426316, 0.10433767263157896, 0.0..."
1,6860,"[-0.15436432316550003, 0.09376183200000002, -0..."
2,6862,"[-0.15436432316550003, 0.09376183200000002, -0..."
3,6863,"[-0.15436432316550003, 0.09376183200000002, -0..."
4,6864,"[-0.15436432316550003, 0.09376183200000002, -0..."
...,...,...
157,7344,"[-0.1489323275426316, 0.10433767263157896, 0.0..."
158,7345,"[-0.12379224302833336, 0.07958752544444445, 0...."
159,7346,"[-0.1489323275426316, 0.10433767263157896, 0.0..."
160,7347,"[-0.09352477003857142, 0.08526532771428572, 0...."


In [35]:
!pip install scikit-learn
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
df_final.head()

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,preprocessed,norm,embedding
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.07638274, 0.15438885, 0.2027531, 0.15020091..."
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.07638274, 0.15438885, 0.2027531, 0.15020091..."
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.07638274, 0.15438885, 0.2027531, 0.15020091..."
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.07638274, 0.15438885, 0.2027531, 0.15020091..."
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,курс «философия» даёт целостное представление ...,"['курс', 'философия', 'давать', 'целостный', '...","[0.07638274, 0.15438885, 0.2027531, 0.15020091..."


In [45]:
def calculate_similarity(row):
  return cosine_similarity(
      [row['embedding']],
      [grouped[grouped['up_id'] == row['up_id']]['emb_up'].values[0]]
      )
  [0][0]

unique_up_ids = df_final['up_id'].unique()[:10]
plans = df_final[df_final['up_id'].isin(unique_up_ids)]
plans['similarity'] = plans.apply(calculate_similarity, axis=1)
plans = plans.sort_values(by='similarity', ascending=False)
plans.head()

<ipython-input-45-e152cc3558df>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plans['similarity'] = plans.apply(calculate_similarity, axis=1)


,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,preprocessed,norm,embedding,similarity
1629,2700,Web-программирование,bachelor,14.0,6974,Информатика и программирование,2020,"Специализация ""Программирование""",В рамках данной двухсеместровой дисциплины буд...,в рамках данной двухсеместровой дисциплины буд...,"['рамка', 'дать', 'двухсеместровый', 'дисципли...","[-0.1799708, 0.10193519, -0.03121634, 0.223620...",[[0.8697176584700218]]
1626,2698,Распределенное программирование,bachelor,14.0,6974,Информатика и программирование,2020,"Специализация ""Программирование""",Дисциплина посвящена изучению современных мето...,дисциплина посвящена изучению современных мето...,"['дисциплина', 'посвятить', 'изучение', 'совре...","[-0.04279694, -0.02472726, 0.15468136, 0.14086...",[[0.8620081988680668]]
1630,2709,Квантовая оптика,bachelor,2.0,6929,Информатика и программирование,2019,"Специализация ""Математическое моделирование""",Данный курс создан для подготовки студентов к ...,данный курс создан для подготовки студентов к ...,"['данный', 'курс', 'создать', 'подготовка', 'с...","[-0.0323108407, 0.0803476826, 0.0813589926, 0....",[[0.857411245162589]]
1624,2698,Распределенное программирование,bachelor,14.0,6881,Информатика и программирование,2018,"Специализация ""Программирование""",Дисциплина посвящена изучению современных мето...,дисциплина посвящена изучению современных мето...,"['дисциплина', 'посвятить', 'изучение', 'совре...","[-0.04279694, -0.02472726, 0.15468136, 0.14086...",[[0.8538666828180358]]
1625,2698,Распределенное программирование,bachelor,14.0,6929,Информатика и программирование,2019,"Специализация ""Программирование""",Дисциплина посвящена изучению современных мето...,дисциплина посвящена изучению современных мето...,"['дисциплина', 'посвятить', 'изучение', 'совре...","[-0.04279694, -0.02472726, 0.15468136, 0.14086...",[[0.8478321031794339]]


In [47]:
similarity_threshold = 0.5
grouped_plans = plans.groupby('up_id')
for plan_id, group_data in grouped_plans:
    plan_name = group_data['up_title'].iloc[0]
    similarity_values = group_data['similarity'].values
    outliers_count = (similarity_values < similarity_threshold).sum()
    print(f"Учебный план '{plan_name}' (ID {plan_id}) содержит {outliers_count} выбивающихся дисциплин.")

Учебный план 'Нанофотоника и квантовая оптика' (ID 6859) содержит 7 выбивающихся дисциплин.
Учебный план 'Информационные технологии в энергетике' (ID 6860) содержит 9 выбивающихся дисциплин.
Учебный план 'Информатика и программирование' (ID 6881) содержит 4 выбивающихся дисциплин.
Учебный план 'Программирование и интернет-технологии' (ID 6905) содержит 4 выбивающихся дисциплин.
Учебный план 'Информатика и программирование' (ID 6929) содержит 6 выбивающихся дисциплин.
Учебный план 'Программирование и интернет-технологии' (ID 6938) содержит 7 выбивающихся дисциплин.
Учебный план 'Информатика и программирование' (ID 6974) содержит 5 выбивающихся дисциплин.
Учебный план 'Программирование и интернет-технологии' (ID 7014) содержит 5 выбивающихся дисциплин.
Учебный план 'Информатика и программирование' (ID 7136) содержит 4 выбивающихся дисциплин.
Учебный план 'Программирование и интернет-технологии' (ID 7146) содержит 5 выбивающихся дисциплин.


In [74]:
plans[plans['up_id'] == 6881]

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,preprocessed,norm,embedding,similarity
1624,2698,Распределенное программирование,bachelor,14.0,6881,Информатика и программирование,2018,"Специализация ""Программирование""",Дисциплина посвящена изучению современных мето...,дисциплина посвящена изучению современных мето...,"['дисциплина', 'посвятить', 'изучение', 'совре...","[-0.04279694, -0.02472726, 0.15468136, 0.14086...",[[0.8538666828180358]]
952,2656,Математическое и имитационное моделирование в ...,bachelor,14.0,6881,Информатика и программирование,2018,Естественнонаучный модуль,В рамках дисциплины студенты знакомятся с осно...,в рамках дисциплины студенты знакомятся с осно...,"['рамка', 'дисциплина', 'студент', 'знакомитьс...","[-0.19465915, 0.12911162, 0.20029833, 0.371702...",[[0.8291599830820373]]
982,2661,Проектирование информационных систем,bachelor,14.0,6881,Информатика и программирование,2018,Элективный модуль по группе направлений,Дисциплина направлена на получение знаний по о...,дисциплина направлена на получение знаний по о...,"['дисциплина', 'направить', 'получение', 'знан...","[-0.2241048, -0.21174316, 0.40210425, 0.364829...",[[0.7904406224446192]]
976,2660,Проектирование программного обеспечения,bachelor,14.0,6881,Информатика и программирование,2018,Элективный модуль по группе направлений,Дисциплина направлена на формирование у студен...,дисциплина направлена на формирование у студен...,"['дисциплина', 'направить', 'формирование', 'с...","[-0.12078195, -0.0632166305, 0.156491532, 0.28...",[[0.7677593670073759]]
1606,2676,Телекоммуникационные системы и технологии,bachelor,14.0,6881,Информатика и программирование,2018,Факультетский модуль,Курс «Телекоммуникационные системы и технологи...,курс «телекоммуникационные системы и технологи...,"['курс', 'телекоммуникационный', 'система', 'т...","[-0.23772338, 0.2000528, -0.35497513, -0.01998...",[[0.7365758632569608]]
381,2642,Методы и средства программного обеспечения,All_levels,12.0,6881,Информатика и программирование,2018,ОГНП 3: Компьютерные технологии,Дициплина предназначена для изучения студентам...,дициплина предназначена для изучения студентам...,"['дициплина', 'предназначить', 'изучение', 'ст...","[-0.134503855, 0.115223753, 0.161448596, 0.470...",[[0.7356986953721474]]
11,2631,Современные вопросы информатики,bachelor,14.0,6881,Информатика и программирование,2018,Модуль «Цифровая культура»,Дисциплина направлена на формирование системат...,дисциплина направлена на формирование системат...,"['дисциплина', 'направить', 'формирование', 'с...","[-0.11285775, 0.01766124, 0.16012359, 0.139779...",[[0.7223121347934823]]
966,2659,Операционные системы,bachelor,14.0,6881,Информатика и программирование,2018,Общепрофессиональный модуль,Дисциплина сконцентрирована на восприятии опер...,дисциплина сконцентрирована на восприятии опер...,"['дисциплина', 'сконцентрировать', 'восприятие...","[-0.194243132, -0.0767890631, -0.121830763, 0....",[[0.7204461822294059]]
7,2628,Проектирование баз данных,bachelor,14.0,6881,Информатика и программирование,2018,Модуль «Цифровая культура»,Дисциплина направлена на формирование понимани...,дисциплина направлена на формирование понимани...,"['дисциплина', 'направить', 'формирование', 'п...","[-0.378083253, 0.102053111, 0.414677315, 0.357...",[[0.6916852762532782]]
501,2643,Основы кибербезопасности,bachelor,13.0,6881,Информатика и программирование,2018,ОГНП 3: Компьютерные технологии,В данном курсе студенты ознакомятся с местом и...,в данном курсе студенты ознакомятся с местом и...,"['данный', 'курс', 'студент', 'ознакомиться', ...","[-0.39175237, -0.30281395, 0.15287935, 0.11740...",[[0.6690999686860238]]


### 3.4.2 Генерация содержания

В этой части работы вам предлагается повторить пайплайн дообучения модели [LLaMA](https://github.com/facebookresearch/llama) (LLM с открытым исходным кодом) из туториала: https://colab.research.google.com/drive/1tG9eqttfnqHoQqmsiacywUG9ilUhoiCk?usp=sharing#scrollTo=qL5i358PRGgu

С текстовым описанием: https://www.datacamp.com/tutorial/fine-tuning-llama-2

Если памяти или ресурсов колаба не хватит, то попробуйте без дообучения. Но результат будет хуже.

Выберите 10-15 дисциплин (случайных или интересных вам) и с помощью модели сгенерируйте возможное описание дисциплины.

Постройте эмбеддинг для полученного описания. Оцените близость полученного эмбеддинга и того, что у вас уже был с момента создания word2vec.



In [60]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [61]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [68]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
base_model = "NousResearch/Llama-2-7b-chat-hf"
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

Looking in indexes: https://pypi.org/simple/


In [67]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [75]:
#псевдокод
courses = plans[plans['up_id'] == 6881]['course_title'].tolist()

generated_descriptions = {}
for course in courses:
  generated_description = model.generate_description(course)
  generated_descriptions[course] = generated_description

generated_embeddings = {}
for course, generated_description in generated_descriptions.items():
  generated_embedding = model.get_embedding(generated_description)
  generated_embeddings[course] = generated_embeddings

selected_embeddings = plans[plans['up_id'] == 6881]['embedding'].tolist()
previous_embeddings = dict(zip(courses, selected_embeddings))
similarity_scores = {}
for course, embedding in generated_embeddings.items():
  if course in previous_embeddings:
    previous_embedding = previous_embeddings[course]
    similarity = cosine_similarity([embedding], [previous_embedding])
    similarity_scores[course] = similarity[0][0]

for course, score in similarity_scores.items():
  print(f"Дисциплина: {course}, Близость: {score}")

AttributeError: 'Word2Vec' object has no attribute 'generate_description'

### Дополнительно (необязательно)

Среди выбранных вами дисциплин найдите те, которые реализуются больше одного года. Изучите их следующим образом:

1. Оцените, насколько содержательно близки дисциплины разных лет.
2. Оцените, к дисциплине какого года ближе всего сгенерированное содержание.